# 텍스트 분류 4차 시도 (불용어 50개 적용) 

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from konlpy.tag import Okt, Kkma, Mecab, Hannanum
Okt = Okt()
kkma = Kkma()
hannanum = Hannanum()

In [3]:
df = pd.read_csv('recent_data_V3.csv', sep='\t', encoding='UTF-8')

In [4]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인', '검토결과'],
      dtype='object')

In [5]:
df.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   
3               W  2018     20180928         ZZ7         B11  조립 문제_미장착/오장착   
4               W  2016     20160526         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   
3         KV11          YP     A9          TT        83120   
4         KV11          YP     A9          TT        83860   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  \
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN   
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN   
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN   
3              [C] 현상및조치   차량출고시 웨즈스트립고무 주름발생 들뜸교환조치     X  NaN  NaN  NaN   
4                     [C] 몰딩 내부 고무 미조립 상태에서 몰딩 조립 출고     X  NaN  NaN  NaN   

   검토결과  
0  X---  
1  X---  
2  X---  
3  X---  
4  X---

In [6]:
df.shape

(10629, 17)

In [7]:
df_O = df.loc[df['대상 선정']=='O']
df_X = df.loc[df['대상 선정']=='X']
df2 = pd.concat([df_O,df_X])

In [8]:
# 제거
# df = df[df['대상 선정'].notnull()].copy()
# 다른값으로 채우기
df = df.fillna('N')

In [9]:
df_N = df.loc[df['대상 선정']=='N']

In [10]:
df2.shape

(8532, 17)

In [11]:
df_N.shape

(2097, 17)

In [12]:
df2 = df2.reindex()
df_N = df_N.reindex()

# 분류현황

In [13]:
category_count = df2['대상 선정'].value_counts()
category_count

X    5519
O    3013
Name: 대상 선정, dtype: int64

In [16]:
category_count = df2['구분'].value_counts()
category_count

기능(커넥터)    1601
실내부품이음     1050
하체이음        361
Name: 구분, dtype: int64

In [14]:
category_count = df2['책임구'].value_counts()
category_count

구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [15]:
category_count = df2['원인'].value_counts()
category_count

부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

# 전처리

In [17]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df2['target_pred'] = df2['대상 선정'].astype(str).copy()
# df2 ['train_charge_predict'] = df2 ['책임구'].astype(str).copy()
# df2 ['train_cause_predict'] = df2 ['원인'].astype(str).copy()

In [18]:
df_N['target_pred'] = df_N['대상 선정'].astype(str).copy()

In [19]:
def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    #text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [20]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

In [21]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(preprocessing)

Wall time: 125 ms


In [22]:
%time df2['clm_prtl_sbc'] = df2['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 234 ms


In [23]:
df2['clm_prtl_sbc'].head(10)

12          가  인도후 오디오 작동시 주파수 및 노이즈 발생으로 불만 제기   및 ...
14          주행중 엔진경고등 점등   및  스캔점검결과 P       DPF효율저하...
15      테일게이트트림에서 뜨르르르 이상소음발생으로 테일게이트트림 탈착하여 확인시 클립이...
19                     조수석사이드에어백 경고등 점등됨 컨넥터 체결불량으로 재체결
24                        USB 작동 안됨원인 USB 컨넥터 조립 않됨 재조립
25                       어라운드뷰 장착차량인데 운전석 미러에는 카메라 미장착됨
32        리어LH윈도우작동불량  점검시 뒤운전석쪽만 작동불량 퓨즈확인시정상   트림탈...
38      운전석 뒤도어 유리창 작동 안되어  메인파워스위치 배선인가 점검후뒷도어트림스위치...
40       상기차량 주행중 운전석 시트가 들석이는 현상발생입고 점검시 운전석 앞 우측 시...
58           와이퍼 스몰등 후방카메라 오작동 불량   및   리어 루프 배선 R  ...
Name: clm_prtl_sbc, dtype: object

In [24]:
%time df_N['clm_prtl_sbc'] = df_N['clm_prtl_sbc'].apply(preprocessing)

Wall time: 31.2 ms


In [25]:
%time df_N['clm_prtl_sbc'] = df_N['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 62.4 ms


In [26]:
df_N['clm_prtl_sbc'].head(10)

224         전원 안꺼지는 현상으로 입고    전원이 안꺼져 베터리 단자 탈거 한 상태...
225      위 콘솔 암레스트 커버 문제로 입고되어 점검후 콘손 암레스트 커버 가죽이 접착불...
226         R H슬라이딩도어 작동불량으로 입고  점검  점검결과 L H슬라이딩 도어 ...
227       열시트열선 작동불량점검내용 열시트 작동불량으로 입고확인시 리어 열선휴즈 소손확...
228        리어스포일러 우측 들뜸발생으로 입고 점검  점검결과 스포일러 조립시 넛트 미...
229                                     커버 미장착으로 부품조립 작업
230       리어도어 힌지 고정볼트 미조립 출고됨점검 및  점검결과 생산과정에서 리어도어 ...
231           엔진경고등 점등으로 입고됨    점검   스캔장비 이용하여 고장코드 검...
232         시동 및 주행시 소음발생으로 입고   확인시 센터 머플러 너트 체결되어있지...
233          리어 도어 위도우 트림판넬 커버 미 장착되어 차량입고   점검  점검결과...
Name: clm_prtl_sbc, dtype: object

# 학습데이터와 테스트데이터 만들기

In [28]:
df2.shape

(8532, 18)

In [27]:
df_N.shape

(2097, 18)

In [29]:
df_train = df2.copy()
df_train.shape

(8532, 18)

In [30]:
df_test = df_N.copy()
df_test.shape

(2097, 18)

In [51]:
df_test

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd  \
224                 W  2017     20170731         ZZ7         B11   
225                 W  2016     20160617         ZZ3         B11   
226                 W  2015     20150129         ZZ7         B11   
227                 W  2017     20171026         ZZ5         B11   
228                 W  2017     20170920         ZZ5         B11   
229                 W  2017     20171115         ZZ5         B11   
230                 W  2017     20170926         ZZ7         B11   
231                 W  2018     20181226         ZZ7         B11   
232                 W  2016     20160823         ZZ7         B11   
233                 W  2017     20170612         ZZ3         B11   
234                 W  2018     20180427         ZZ5         B11   
235                 W  2016     20160727         ZZ7         B11   
236                 W  2015     20150720         ZZ7         B11   
237                 W  2018     20180208         ZZ7         B11   
238                 W  2015     20150604         ZZ5         B11   
239                 W  2016     20160325         ZZ5         B11   
240                 W  2018     20181115         ZZ5         B11   
241                 W  2018     20180620         ZZ5         B11   
242                 W  2015     20150731         ZZ5         B11   
243                 W  2015     20150305         ZZ3         B11   
244                 W  2017     20170207         ZZ5         B11   
245                 W  2015     20151113         ZZ3         B11   
246                 W  2016     20161026         ZZ7         B11   
247                 W  2018     20180522         ZZ7         B11   
248                 W  2016     20160720         ZZ5         B11   
249                 W  2015     20150904         ZZ5         B11   
250                 W  2017     20171012         ZZ1         B11   
251                 W  2018     20181112         ZZ4         B11   
252                 W  2015     20150601         ZZ7         B11   
253                 W  2015     20151113         ZZ7         B11   
...               ...   ...          ...         ...         ...   
10522               W  2016     20160113         ZZ3         V91   
10523               W  2015     20151027         ZZ5         V91   
10524               W  2015     20150311         ZZ5         V91   
10525               W  2018     20180519         ZZ3         V91   
10526               W  2018     20180713         ZZ2         V91   
10527               W  2016     20160122         ZZ4         V91   
10528               W  2016     20160310         ZZ3         V91   
10529               W  2017     20171031         ZZ3         V91   
10530               W  2017     20170126         ZZ5         V91   
10531               W  2016     20160204         ZZ3         V91   
10532               W  2016     20160126         ZZ4         V91   
10533               W  2015     20150310         ZZ5         V91   
10606               W  2016     20160202         ZZ5         V92   
10607               W  2016     20160310         ZZ5         V92   
10608               W  2016     20160204         ZZ5         V92   
10609               W  2015     20150924         ZZ3         V92   
10610               W  2016     20161130         ZZ3         V92   
10611               W  2015     20150414         ZZ3         V92   
10612               W  2018     20180213         ZZ5         V92   
10613               W  2016     20160115         ZZ8         V92   
10614               W  2015     20150226         ZZ5         V92   
10615               W  2016     20160212         ZZ5         V92   
10616               W  2017     20170223         ZZ5         V92   
10617               W  2015     20151229         ZZ3         V92   
10618               W  2015     20150826         ZZ5         V92   
10619               W  2016     20160325         ZZ5         V92   
10620               W  2016     20160216         ZZ3         V

# 단어 벡터화

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

stops = ['[C]']
vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor = None, # 전처리 도구
                             stop_words = stops, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
                             min_df = 1, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3), # BOW의 단위를 1~3개로 지정합니다.
                             max_features = 20000 # 만들 피처의 수, 단어의 수가 된다.
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None, stop_words=['[C]'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [32]:
#%%time 
train_feature_vector = vectorizer.fit_transform(df_train['clm_prtl_sbc'])
train_feature_vector.shape

(8532, 20000)

In [33]:
#%%time 
test_feature_vector = vectorizer.fit_transform(df_test['clm_prtl_sbc'])
test_feature_vector.shape

(2097, 20000)

In [34]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

20000


['abs',
 'abs 경고등',
 'abs모듈',
 'abs센서',
 'acc상태유지',
 'adm',
 'ae',
 'aeb',
 'aeb 경고등',
 'af']

In [35]:
dist = np.sum(train_feature_vector, axis=0)

pd.DataFrame(dist, columns=vocab)

abs  abs 경고등  abs모듈  abs센서  acc상태유지  adm  ae  aeb  aeb 경고등  af ...  히터 에어컨  \
0   56        4      5      3       11    4   4    6        3   4 ...      20   

   히터 온도  히터 제어  히터 컨트롤  히터제어  히터제어 작동범위  히터컨트롤  히터회로  히트  힌지  
0      4      3       3     4          9      3     5  26   3  

[1 rows x 20000 columns]

# tf-idf 가중치 적용

In [36]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [37]:
#%%time 
train_feature_tfidf = transformer.fit_transform(train_feature_vector)
train_feature_tfidf.shape

(8532, 20000)

In [38]:
#%%time 
test_feature_tfidf = transformer.fit_transform(test_feature_vector)
test_feature_tfidf.shape

(2097, 20000)

# 머신러닝 : 학습

In [39]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [40]:
# 학습에 사용할 y_label 을 넣어줍니다.
# 어떤 분야의 청원인지 예측할 것이기 때문에 category를 넣어줍니다.
# y_label2 = df_train['caus_pred']
y_label = df_train['target_pred']
# y_label3 = df_train['charge_pred']
%time forest = forest.fit(train_feature_tfidf, y_label)
# %time forest2 = forest.fit(train_feature_tfidf, y_label2)
# %time forest3 = forest.fit(train_feature_tfidf, y_label3)

Wall time: 2.42 s


# 모델평가

In [41]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, train_feature_vector, y_label, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 34.9 s


array([0.75219684, 0.7410662 , 0.78604924, 0.75439625, 0.74091442])

In [42]:
round(np.mean(score)*100,2)

75.49

# 예측 → testdata에 ox에 대한 값이 미입력되어서 예측값 결과검증은 불가

In [43]:
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(test_feature_vector)
y_pred[:3]

array(['X', 'X', 'X'], dtype=object)

In [44]:
y_pred.shape

(2097,)

In [45]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 줍니다.
output = pd.DataFrame(data={'category_pred':y_pred})
output.head(10)

category_pred
0             X
1             X
2             X
3             O
4             X
5             X
6             O
7             O
8             X
9             X

In [46]:
df_test['category_pred'] = y_pred

In [50]:
df_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
224               W  2017     20170731         ZZ7         B11  조립 문제_미장착/오장착   
225               W  2016     20160617         ZZ3         B11  조립 문제_미장착/오장착   
226               W  2015     20150129         ZZ7         B11  조립 문제_미장착/오장착   
227               W  2017     20171026         ZZ5         B11  조립 문제_미장착/오장착   
228               W  2017     20170920         ZZ5         B11  조립 문제_미장착/오장착   

    trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
224         KV11          YP     A9          CC        43777   
225         KV11          YP     A9          TT        84660   
226         KV11          YP     A9          TT        82940   
227         KV11          YP     A9          AC        91503   
228         KV11          CK     J5          EE        13271   

                                          clm_prtl_sbc 대상 선정 구분 책임구 원인 검토결과  \
224       전원 안꺼지는 현상으로 입고    전원이 안꺼져 베터리 단자 탈거 한 상태...     N  N   N  N    N   
225    위 콘솔 암레스트 커버 문제로 입고되어 점검후 콘손 암레스트 커버 가죽이 접착불...     N  N   N  N    N   
226       R H슬라이딩도어 작동불량으로 입고  점검  점검결과 L H슬라이딩 도어 ...     N  N   N  N    N   
227     열시트열선 작동불량점검내용 열시트 작동불량으로 입고확인시 리어 열선휴즈 소손확...     N  N   N  N    N   
228      리어스포일러 우측 들뜸발생으로 입고 점검  점검결과 스포일러 조립시 넛트 미...     N  N   N  N    N   

    target_pred category_pred  pred_diff  
224           N             X          0  
225           N             X          0  
226           N             X          0  
227           N             O          0  
228           N             X          0